In [1]:
## import libraries
import os, sys
import yaml
import xarray as xr
import pandas as pd
import numpy as np
import metpy.calc as mpcalc
from metpy.units import units
import dask
from datetime import timedelta
import glob
import itertools
%matplotlib inline
import seaborn as sns

# plotting
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import cartopy.crs as ccrs
from matplotlib.gridspec import GridSpec
from matplotlib.colorbar import Colorbar # different way to handle colorbar
import textwrap

import matplotlib as mpl
mpl.use('agg')

# # import personal modules
# sys.path.append('../modules')
# import ar_funcs

dask.config.set(**{'array.slicing.split_large_chunks': True})

sys.path.append('../../mclimate_tool_cw3e')
from build_html_table import create_html_table
import custom_cmaps as ccmap
from plotter import draw_basemap, plot_mclimate_forecast, plot_mclimate_forecast_four_panel
# from plotter import draw_basemap

sys.path.append('../modules')
import customcmaps as ccmaps

import mclimate_funcs as mclim_func

In [2]:
path_to_data = '/expanse/nfs/cw3e/cwp140/'      # project data -- read only
path_to_out  = '../out/'       # output files (numerical results, intermediate datafiles) -- read & write
path_to_figs = '../figs/'      # figures

In [3]:
### list of dates currently ran
## '20051120' 'GEFSv12_reforecast' '20051124'
## '20070607' 'GEFSv12_reforecast'
## '20150813' 'GEFSv12_reforecast' '20150818'
## '20201128' 'GEFS_archive' '20201202'
## '20231117' 'GEFS_archive''20231120'
## '20240919' 'GEFS_archive' '20240923'

In [ ]:
## TODO write a script to create dictionary for above dates
## plus add in landslide dates from Eliza's database
## include small script that creates list of Fdate (init dates) 
## every 24 hour for 0-10 days before impact date

## then add loop for the code below 
## to run multiple init dates for each impact date
## then make this into a .py to use with the above dictionary

## then we want to keep the max percentile information 
## for day of impact for each init date
## save to csv

In [4]:
%%time
######################
### VARS TO UPDATE ###
######################
fdate = '20240919' ## initialization date in YYYYMMDD format
fdate2 = '20240923' ## date of impact
model = 'GEFS_archive' ## 'GEFSv12_reforecast', 'GFS', 'GEFS', 'GEFS_archive'
map_ext = [-170., -120., 40., 65.] ## map extent [minlon, maxlon, minlat, maxlat]
# [-170., -125., 50., 75.] AK
# [-170., -120., 50., 75.] SEAK/NPAC
table_ext = [-141., -130., 54.5, 60.] ## extent to choose the maximum value from for the table [minlon, maxlon, minlat, maxlat]
# [-141., -130., 54.5, 60.] SEAK
# [-160., -145., 65., 70.] Northern AK
fig_path = path_to_figs + 'images_historical/mclimate_{0}/'.format(fdate)
os.makedirs(os.path.dirname(fig_path), exist_ok=True)

####################################
### COMPARE FORECAST TO MCLIMATE ###
####################################
var_lst = ['ivt', 'freezing_level', 'uv1000']
ds_lst = []
ds_lst2 = []
fc_lst = []
for i, varname in enumerate(var_lst):
    forecast, ds = mclim_func.run_compare_mclimate_forecast(varname, fdate, model, server='expanse')
    fc_lst.append(forecast)
    ds_lst.append(ds)

    if varname == 'uv1000':
            ds = ds.rename({'mclimate': 'uv'})
    else:
        ds = ds.rename({'mclimate': varname})
    ds_lst2.append(ds)
    
### merge the datasets
ds3 = xr.merge(ds_lst2)
ds3 = ds3.sortby('lat')

fc = xr.merge(fc_lst)
fc = fc.sortby('lat')

09 19
09 19
09 19
CPU times: user 3.58 s, sys: 1.37 s, total: 4.95 s
Wall time: 18.6 s


In [5]:
fc

<xarray.Dataset>
Dimensions:         (lat: 121, step: 40, lon: 140)
Coordinates:
    init_date       datetime64[ns] 2024-09-19
    surface         float64 0.0
  * lat             (lat) float64 10.0 10.5 11.0 11.5 ... 68.5 69.0 69.5 70.0
    valid_time      (step) datetime64[ns] 2024-09-19T06:00:00 ... 2024-09-29
  * lon             (lon) float64 -179.5 -179.0 -178.5 ... -111.0 -110.5 -110.0
  * step            (step) int64 6 12 18 24 30 36 42 ... 210 216 222 228 234 240
    isothermZero    float64 0.0
    isobaricInhPa   float64 1e+03
Data variables:
    ivtu            (step, lat, lon) float64 -489.7 -501.7 ... 32.54 32.63
    ivtv            (step, lat, lon) float64 -119.5 -114.8 ... -6.814 -7.871
    ivt             (step, lat, lon) float64 504.0 514.7 523.2 ... 33.25 33.56
    freezing_level  (step, lat, lon) float32 5.216e+03 5.193e+03 ... 363.4 365.8
    u               (step, lat, lon) float32 -5.64 -5.75 -6.92 ... 0.87 0.78
    v               (step, lat, lon) float32 -3.79 -3.94 -3.13 ... -1.16 -1.24
    uv              (step, lat, lon) float32 6.795 6.97 7.595 ... 1.45 1.465

In [6]:
## compute duration of IVT >= 95th percentile
AR = xr.where(ds3.ivt >= 0.95, 1, 0)
a = AR != 0 # this will place True for all rows where AR is not 0

# get the temporal resolution in hours
t = ds3['step'].isel(step=1) - ds3['step'].isel(step=0) 
nhrs = t.values.astype('timedelta64[h]') # convert to hours

## this grabs the start and stop indices of each AR
tmp = a.cumsum(dim='step')-a.cumsum(dim='step').where(~a).ffill(dim='step').fillna(0).astype(int) # cumulative sum where not 0
duration = tmp*nhrs.astype(int)
duration = duration.rename("duration")
ds3 = xr.merge([ds3, duration])
ds3

<xarray.Dataset>
Dimensions:         (step: 40, lat: 121, lon: 140)
Coordinates:
  * lat             (lat) float64 10.0 10.5 11.0 11.5 ... 68.5 69.0 69.5 70.0
  * lon             (lon) float64 -179.5 -179.0 -178.5 ... -111.0 -110.5 -110.0
  * step            (step) int64 6 12 18 24 30 36 42 ... 210 216 222 228 234 240
    init_date       datetime64[ns] 2024-09-19
    surface         float64 0.0
    isothermZero    float64 0.0
    isobaricInhPa   float64 1e+03
Data variables:
    ivt             (step, lat, lon) float64 0.94 0.95 0.95 0.96 ... nan nan nan
    freezing_level  (step, lat, lon) float64 0.99 0.98 0.97 0.95 ... nan nan nan
    uv              (step, lat, lon) float64 nan nan nan 0.75 ... nan nan nan
    duration        (step, lat, lon) int64 0 6 6 6 6 6 6 6 6 ... 0 0 0 0 0 0 0 0

In [7]:
## calculate index score
## +1 Index Point for IVT >= 95th percentile
AR1 = xr.where(ds3.ivt >= 0.95, 1, 0)

## +1 Index Point for freezing_level >= 95th percentile
AR2 = xr.where(ds3.freezing_level >= 0.95, 1, 0)

## +1 Index Point for uv1000 >= 95th percentile
AR3 = xr.where(ds3.uv >= 0.95, 1, 0)

## +1 Index Point for duration >= 24
AR4 = xr.where(ds3.duration >= 24, 1, 0)

## +1 Index Point for duration >= 48
AR5 = xr.where(ds3.duration >= 48, 1, 0)

AR_index = AR1 + AR2 + AR3 + AR4 + AR5

AR_index = AR_index.rename("AR_index")
ds3 = xr.merge([ds3, AR_index])
ds3

<xarray.Dataset>
Dimensions:         (step: 40, lat: 121, lon: 140)
Coordinates:
  * lat             (lat) float64 10.0 10.5 11.0 11.5 ... 68.5 69.0 69.5 70.0
  * lon             (lon) float64 -179.5 -179.0 -178.5 ... -111.0 -110.5 -110.0
  * step            (step) int64 6 12 18 24 30 36 42 ... 210 216 222 228 234 240
    init_date       datetime64[ns] 2024-09-19
    surface         float64 0.0
    isothermZero    float64 0.0
    isobaricInhPa   float64 1e+03
Data variables:
    ivt             (step, lat, lon) float64 0.94 0.95 0.95 0.96 ... nan nan nan
    freezing_level  (step, lat, lon) float64 0.99 0.98 0.97 0.95 ... nan nan nan
    uv              (step, lat, lon) float64 nan nan nan 0.75 ... nan nan nan
    duration        (step, lat, lon) int64 0 6 6 6 6 6 6 6 6 ... 0 0 0 0 0 0 0 0
    AR_index        (step, lat, lon) int64 1 2 2 2 1 1 1 1 2 ... 0 0 0 0 0 0 0 0

In [13]:
####################
### CREATE PLOTS ###
####################
# region_lst = ['NPAC']
# step_lst = ds3.step.values
# print('...Writing plots')
# for i, (varname, ds, fc) in enumerate(zip(var_lst, ds_lst, fc_lst)):
#     for region in region_lst:
#         for step in step_lst:
#             print('...for {0}, {1}, {2}'.format(varname, region, step))
#             out_fname = fig_path + '{2}_{0}_mclimate_F{1}'.format(varname, step, region)
#             ## write plots
#             plot_mclimate_forecast(ds, fc, step=step, varname=varname, fname=out_fname, ext_name=region, historical=True)

step_lst = ds3.step.values
for i, step in enumerate(step_lst[15:19]):
    print(step)
    out_fname = fig_path + 'SEAK_mclimate_F{0}'.format(step)
    plot_mclimate_forecast_four_panel(ds3, fc, step, out_fname, domain="SEAK")
    out_fname = fig_path + 'NPAC_mclimate_F{0}'.format(step)
    plot_mclimate_forecast_four_panel(ds3, fc, step, out_fname, domain="NPAC")

96


/home/dnash/miniconda3/envs/SEAK-impacts/lib/python3.12/site-packages/matplotlib/quiver.py:627: RuntimeWarning: Mean of empty slice.
  amean = a[~self.Umask].mean()
/home/dnash/miniconda3/envs/SEAK-impacts/lib/python3.12/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


102


/home/dnash/miniconda3/envs/SEAK-impacts/lib/python3.12/site-packages/matplotlib/quiver.py:627: RuntimeWarning: Mean of empty slice.
  amean = a[~self.Umask].mean()
/home/dnash/miniconda3/envs/SEAK-impacts/lib/python3.12/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/home/dnash/miniconda3/envs/SEAK-impacts/lib/python3.12/site-packages/matplotlib/quiver.py:627: RuntimeWarning: Mean of empty slice.
  amean = a[~self.Umask].mean()
/home/dnash/miniconda3/envs/SEAK-impacts/lib/python3.12/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


108


/home/dnash/miniconda3/envs/SEAK-impacts/lib/python3.12/site-packages/matplotlib/quiver.py:627: RuntimeWarning: Mean of empty slice.
  amean = a[~self.Umask].mean()
/home/dnash/miniconda3/envs/SEAK-impacts/lib/python3.12/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/home/dnash/miniconda3/envs/SEAK-impacts/lib/python3.12/site-packages/matplotlib/quiver.py:627: RuntimeWarning: Mean of empty slice.
  amean = a[~self.Umask].mean()
/home/dnash/miniconda3/envs/SEAK-impacts/lib/python3.12/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


114


/home/dnash/miniconda3/envs/SEAK-impacts/lib/python3.12/site-packages/matplotlib/quiver.py:627: RuntimeWarning: Mean of empty slice.
  amean = a[~self.Umask].mean()
/home/dnash/miniconda3/envs/SEAK-impacts/lib/python3.12/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/home/dnash/miniconda3/envs/SEAK-impacts/lib/python3.12/site-packages/matplotlib/quiver.py:627: RuntimeWarning: Mean of empty slice.
  amean = a[~self.Umask].mean()
/home/dnash/miniconda3/envs/SEAK-impacts/lib/python3.12/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [9]:
## create html table with max value within extent
ext=[-141., -130., 54., 60.]
tmp = ds3.sel(lat=slice(ext[2], ext[3]), lon=slice(ext[0], ext[1]))
maxval = tmp.max(dim=['lat', 'lon']).fillna(0)


df = maxval.to_dataframe()
df = df.drop(['init_date'], axis=1)
df['ivt'] = df['ivt']*100
df['freezing_level'] = df['freezing_level']*100
df['uv'] = df['uv']*100
df = df.rename(columns={"ivt": "IVT", "freezing_level": "Freezing Level", "uv": "UV", "duration": "Duration"})

## create list of valid dates
ts = pd.to_datetime(ds3.init_date.values, format="%Y%m%d%H")
# ts = ds.init_date.values
init_date = ts.strftime('%Y%m%d%H')
init_time = ts.strftime('Initialized: %HZ %d %b %Y')
col2 = []
date_lbl = []
step_lst = ds3.step.values.tolist()
for i, step in enumerate(step_lst):
    ts_valid = ts + timedelta(hours=step)
    HH = ts_valid.strftime('%H')
    if HH == '06':
        date_lbl.append(ts_valid.strftime('%b %d'))
    valid_str = ts_valid.strftime('%H UTC')
    txt = '{0} | F{1}'.format(valid_str, str(step).zfill(3))
    col2.append(txt)
    
df.index = col2

In [10]:
def create_heatmap_annotation_labels(data):
    ## make custom annotation labels
    ## if value < 75, label is '<75'
    ## if value > 99, label is 'MAX'
    
    lbl_lst = []
    for i, d in enumerate(data):
        if d[0] < 75.:
            lbl = '<75'
        elif d[0] > 99.:
            lbl = 'MAX'
        else:
            lbl = "{:.0f}".format(d[0])
        lbl_lst.append(lbl)

    lbl_lst = np.asarray(lbl_lst).reshape(40,1)
    
    return lbl_lst

def create_mini_heatmap(ax, cmap_name, varname, ytcklbl, xtcklbl):
    
    cmap, norm, bnds, cbarticks, cbarlbl = ccmap.cmap(cmap_name)
    data = np.asarray(df[varname]).reshape(40,1)
    if varname != 'Duration':
        data_lbls = create_heatmap_annotation_labels(data)
    else:
        data_lbls = True
        
    sns.heatmap(data, cbar=False, annot=data_lbls, cmap=cmap, norm=norm, linewidths=.5, ax=ax, 
                yticklabels=ytcklbl, xticklabels=[xtcklbl], fmt='')
    ax.xaxis.tick_top()
    ax.set(xlabel="", ylabel="")
    ax.hlines([3, 7, 11, 15, 19, 23, 27, 31, 35, 39], color='k', lw=0.8, *ax.get_xlim())
    plt.yticks(rotation=0)

    return ax

In [11]:
plot_dict = {
  "IVT" : {
    "cmap_name" : "mclimate_green",
    "ytcklbl" : df.index.values,
    "xtcklbl" : 'IVT'
  },
  "Freezing Level" : {
    "cmap_name" : "mclimate_red",
    "ytcklbl" : False,
    "xtcklbl" : 'Z0'
  },
  "UV" : {
    "cmap_name" : "mclimate_purple",
    "ytcklbl" : False,
    "xtcklbl" : 'UV'
  },
  "Duration" : {
    "cmap_name" : "duration",
    "ytcklbl" : False,
    "xtcklbl" : 'DUR'
  },
}

In [12]:
fname = '../figs/heatmap_{0}'.format(fdate)
fmt = 'png'
fig = plt.figure()
fig.set_size_inches((2.0,10.0))
fig.dpi = 300

nrows = 2
ncols = 4
## Use gridspec to set up a plot with a series of subplots that is
## n-rows by n-columns
gs = GridSpec(nrows, ncols, height_ratios=[1, 0.05], width_ratios = [1, 1, 1, 1], wspace=0.05, hspace=0.05)
## use gs[rows index, columns index] to access grids

## loop through each heatmap
varname_lst = ['IVT', 'Freezing Level', 'UV', 'Duration']
for i, varname in enumerate(varname_lst):
    print(varname)
    ax = fig.add_subplot(gs[0, i])
    
    create_mini_heatmap(ax, plot_dict[varname]['cmap_name'], varname, 
                        plot_dict[varname]['ytcklbl'], plot_dict[varname]['xtcklbl'])
    if i == 0:
        ax.set_title(init_time, fontsize=11, loc='left')

        lbl_loc = [1, 4.5, 8.5, 12.5, 16.5, 20.5, 24.5, 28.5, 32.5, 36.5]
        for j, datel in enumerate(date_lbl):
            ## add month day labels
            kw = {'weight': 'bold', 'size': 9}
            ax.text(-3.25, lbl_loc[j]+1.25, textwrap.fill(datel, width=3), va='bottom', ha='center',
                rotation='horizontal', rotation_mode='anchor', **kw)


kw_ticklabels = {'size': 10, 'color': 'dimgray', 'weight': 'light'}
domain = u"{:.0f}\N{DEGREE SIGN}N to {:.0f}\N{DEGREE SIGN}N, {:.0f}\N{DEGREE SIGN}W to {:.0f}\N{DEGREE SIGN}W".format(ext[2], ext[3], ext[0], ext[1])
txt = 'Maximum percentile rank (xth) within {0}'.format(domain)
ann_ax = fig.add_subplot(gs[-1, :])
ann_ax.axis('off')
ann_ax.annotate(textwrap.fill(txt, 40), # this is the text
            (0, 0.), # these are the coordinates to position the label
            textcoords="offset points", # how to position the text
            xytext=(-80,20), # distance from text to points (x,y)
            ha='left', # horizontal alignment can be left, right or center
            **kw_ticklabels)

# # Create a Rectangle patch
# ax.add_patch(mpatches.Rectangle(xy=[-3,0], width=10, height=3,
#                                             fill=False,
#                                             edgecolor='r',
#                                             linewidth=0.75,
#                                             zorder=199))


# Save the figure
fig.savefig('%s.%s' % (fname, fmt), bbox_inches='tight', dpi=fig.dpi)
plt.show()

IVT
Freezing Level
UV
Duration


In [6]:
###################
### BUILD TABLE ###
###################
df = create_html_table(ds3, domain="SEAK")
df_html = df.to_html(index=False, formatters={'Hour': lambda x: '<b>' + x + '</b>'}, escape=False)
df

In [7]:
# ## write to text file
# html = df.to_html(index=False, formatters={'Hour': lambda x: '<b>' + x + '</b>'}, escape=False)

# # write html to file
# text_file = open("../out/table_{0}.html".format(fdate), "w")
# text_file.write(html)
# text_file.close()

In [9]:
#######################
### WRITE HTML FILE ###
#######################
print('...Writing HTML file')
out_fname = "../out/mclimate_tool_historical_{0}.html".format(fdate2)

with open('../../mclimate_tool_cw3e/out/html_text.txt', mode='r') as in_file, \
     open('../../mclimate_tool_cw3e/out/html_text2_{0}.txt'.format(fdate2), mode='r') as in_file2, \
     open(out_fname, mode='w') as out_file:

    # A file is iterable
    # We can read each line with a simple for loop
    for line in in_file:
        out_file.write(line)
        
        
    ## now add in the table
    out_file.write(df_html)

    ## now add the last few lines
    for line in in_file2:
        out_file.write(line)
        
    out_file.close()

...Writing HTML file
